In [409]:
# Source du code qui suit : https://towardsdatascience.com/k-nearest-neighbors-classification-from-scratch-with-numpy-cb222ecfeac1

from sklearn.model_selection import train_test_split
import numpy as np 
import scipy.io as sp


dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']

#Séparation des données d'apprentissage et des données tests
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y)
#Création de la fonction calculant la distance euclidienne entre 2 points
def euclidian_distance(a, b):
    return np.sqrt(np.sum((a-b)**2, axis=1))


#fonction pour trouver les k plus proches voisins de chaque points test
def kneighbors(X_test, return_distance=False): #False si on ne pondère pas par la suite avec les distance, True si on pondère
       
        n_neighbors = 5  #choix de l'hyperparamètre k 
        dist = [] #matrice des distances des knn pour chaque points des données tests
        neigh_ind = [] #matrice des distances des indices des knn pour chaque points des données tests
        
        point_dist = [euclidian_distance(x_test, X_train) for x_test in X_test]  #On stocke les données des distances euclidiennes entre chaque points dans cette matrice
            #chaque ligne correspond à une liste de distances entre un point donné de test et toutes les données d'apprentissage
        
        for row in point_dist:      #On parcours chaque ligne de la matrice des distances pour pouvoir trier les distances par ordre croissant
            enum_neigh = enumerate(row)  #Avant de trier, on énumère chaque ligne pour ne pas perdre les indices des points de données d'apprentissage avec lesquels nous avons calculé les distances
            sorted_neigh = sorted(enum_neigh, key=lambda x: x[1])[:n_neighbors] #Contient les k plus proches voisins de chaque données test, triés en fonction de leurs distances euclidiennes
                        #matrice de type :(index,valeur)
            ind_list = [tup[0] for tup in sorted_neigh]  #on extrait les indices de sorted_neigh
            dist_list = [tup[1] for tup in sorted_neigh]  #on extrait les distances de sorted_neigh
    
            dist.append(dist_list)
            neigh_ind.append(ind_list)
        
        if return_distance: #si on pondère le label avec les distances, on a besoin de connaître les distances au point des 5 voisins
            return np.array(dist), np.array(neigh_ind)
        
        return np.array(neigh_ind)  #si on ne pondère pas avec les points, on a seulement besoin de l'indice du voisin

#print(X_test,X_train,y_test,y_train)

print(kneighbors(X_test, return_distance=False))    

342
[[107 167 197 326   2]
 [ 32  46  61  90 146]
 [114 160 162  91  94]
 ...
 [179 325 339  63  98]
 [242 120  89 316 239]
 [127 305  32  46  61]]


In [410]:
#fonction pour prédire la classe de chaques point test
#cette fonction pondère l'importance de chaque point d'apprentissage en fonction de leur distance au point test

def predict(X_test, weights='uniform'):
        
        class_num = 2
        
        if weights=='uniform': #si on ne pondère pas, donc poids uniforme
            neighbors = kneighbors(X_test)
            y_pred = np.array([np.argmax(np.bincount(np.concatenate(y_train[neighbor]))) for neighbor in neighbors]) #on cherche les indices des voisins, puis on obtient leur label. Chaque ligne de neighbor correspond à l'ensemble des voisins de chaque point de données de test
            #avec bincount, on compte le nombre d'occurence des 0 et des 1 parmi les labels des k voisins de chaque X_test
            #on prend ensuite le label qui à la plus grande occurence parmi les k voisins
            #On réitère pour chaque x_test et on stocke le label prédit dans une matrice        
            return (y_pred)
    
        if weights=='distance': #si on pondère
        
            dist, neigh_ind = kneighbors(X_test, return_distance=True) #On récupère les distances et les indices des k plus proches voisins de chaque X_test
            
            inv_dist = 1/dist
            # le voisin le plus proche du point de données de test a le plus de poids (vote) proportionnel à l'inverse de leurs distances
            
            mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis] # inverse moyen des distances entre voisins
            proba = []
            
            #calcul des probabilités de classe pour chaque point de données de test
            for i, row in enumerate(mean_inv_dist):
                
                row_pred = y_train[neigh_ind[i]] #on récupère le label des k plus proches voisins de chaque X_test
                
                for k in range(class_num):
                    indices = np.where(row_pred==k)
                    prob_ind = np.sum(row[indices[1]]) 
                    proba.append(np.array(prob_ind))
        
            predict_proba = np.array(proba).reshape(X_test.shape[0], class_num)
            
            y_pred = np.array([np.argmax(item) for item in predict_proba])
            
            return y_pred

#print('prédiction avec pondération distance :')
print(predict(X_test, weights='distance'))
print('Y test vrai')
print(np.transpose(y_test))

prédiction avec pondération distance :
[0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1
 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 1 0
 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 1 0 1 0
 0 1 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1
 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0
 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 1 0
 1 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 1 0 1 0 1 0]
Y test vrai
[[0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0
  1 0 0 1 1 0 0 1 0 1 1 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0
  0 1 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 1
  0 1 0 0 1 0 1 0 1 1 0 1 1 0 1 

/var/folders/s6/p3txrvvd4498vs_xw7ryvx7h0000gn/T/ipykernel_28019/866047133.py:20: RuntimeWarning: divide by zero encountered in true_divide
  inv_dist = 1/dist
/var/folders/s6/p3txrvvd4498vs_xw7ryvx7h0000gn/T/ipykernel_28019/866047133.py:24: RuntimeWarning: invalid value encountered in true_divide
  mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis] # inverse moyen des distances entre voisins


In [411]:
#calcul de l'accuracy
#fonctionne pas pour l'instant du coup on utilise la librairie sklearn.metrics 
def score(X_test, y_test):
    y_pred = np.transpose(predict(X_test))
        
    return float(sum(y_pred == np.transpose(y_test)))/ float(len(y_test))

In [412]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predict(X_test, weights='uniform')))
print(accuracy_score(y_test,predict(X_test, weights='distance')))

'''
print('Label prédit Sans pondéradation : ', np.transpose(predict(X_test, weights='uniform')))
print('y_test vrai : ', np.transpose(y_test))
print('Accuracy sans pondération :', score(X_test, y_test))

print('Label prédit Avec pondération', predict(X_test, weights='distance'))
print('Accuracy avec pondération :', score(X_test, y_test))
'''

0.9678362573099415
0.9678362573099415


/var/folders/s6/p3txrvvd4498vs_xw7ryvx7h0000gn/T/ipykernel_28019/866047133.py:20: RuntimeWarning: divide by zero encountered in true_divide
  inv_dist = 1/dist
/var/folders/s6/p3txrvvd4498vs_xw7ryvx7h0000gn/T/ipykernel_28019/866047133.py:24: RuntimeWarning: invalid value encountered in true_divide
  mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis] # inverse moyen des distances entre voisins


"\nprint('Label prédit Sans pondéradation : ', np.transpose(predict(X_test, weights='uniform')))\nprint('y_test vrai : ', np.transpose(y_test))\nprint('Accuracy sans pondération :', score(X_test, y_test))\n\nprint('Label prédit Avec pondération', predict(X_test, weights='distance'))\nprint('Accuracy avec pondération :', score(X_test, y_test))\n"